# Stimuli

In [1]:
import Prelude as P

import Csound.Catalog.Drum.MiniPops (rimShot', rimSpec)
import Csound.Base as C
import Csound.Sam.Chord (chMaj,)
import Csound.Sam as S

import Control.Monad (zipWithM_,)
import Data.Array.Repa as A
import Data.Array.Repa.Eval (fromList,)
import Data.Array.Repa.Repr.Vector (V,)
import Data.Ratio as R

The lower half of the twelve-tone golden scale (G12):

In [2]:
lowerG12 :: Array V DIM1 Rational
lowerG12 = fromList (Z :. 5) [16%15, 10%9, 6%5, 5%4, 4%3]
print $ toList lowerG12

[16 % 15,10 % 9,6 % 5,5 % 4,4 % 3]

Tritonus:

In [8]:
tritonus :: C.D
tritonus = sqrt 2

The upper half of G12:

In [4]:
upperG12 :: Array A.D DIM1 Rational
upperG12 = A.map (2 /) lowerG12
print $ toList upperG12

[15 % 8,9 % 5,5 % 3,8 % 5,3 % 2]

Finally, the entire twelve-tone golden scale:

In [12]:
g12 :: Array V DIM1 C.D
g12 = computeS $ traverse2 lowerG12 upperG12 transExtent newElem
    where transExtent (Z :. i) (Z :. i') = ix1 $ i + 1 + i'
          newElem get get' (Z :. i)
              | i <  5    = fromRational . get  $ ix1 i
              | i == 5    = tritonus
              | otherwise = fromRational . get' $ ix1 i

An infinite click train (60 bpm):

In [3]:
clickTrain = loop . fromSig1 1 $ ticks 1 1

Context/probe length (beats):

In [ ]:
foldAllS lcm 1 . A.map denominator $ 

In [4]:
nbeats = 20

Tempo (bpm):

In [5]:
tempo = 120

Stimulus duration:

In [6]:
duration = 60 * 2 * nbeats / tempo
format ("Each stimulus will be " % float % "s long.") duration

"Each stimulus will be 20s long."

Context:

In [7]:
context = lim nbeats $ mean [str (intvToRat intv) clickTrain | intv <- chMaj]

Probes:

In [8]:
probes = [lim nbeats $ str (intvToRat $ fromIntegral intv) clickTrain | intv <- [0 .. 11]]

Stimuli:

In [9]:
stimuli = [fmap (setDur duration) . runSam tempo $ flow [context, probe] | probe <- probes]
probeLabels = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
fileNames = map (\probeLabel -> show $ format ("< C E G > " % F.text % ".wav") probeLabel) probeLabels
zipWithM_ writeSnd fileNames stimuli